In [1]:
# %%bash
# diamond blastp \
#     -q sarg_ref.fa \
#     -d tmp/refseq_protein.dmnd \
#     --out tmp/refseq_protein.txt \
#     --outfmt 6 qseqid sseqid stitle nident qlen slen pident qcovhsp scovhsp bitscore evalue \
#     --id 50 --subject-cover 75 --query-cover 75 \
#     -k 0 --threads 48 --quiet

In [2]:
import pandas as pd
import glob
import regex as re
import json
import gzip

from collections import defaultdict
from Bio import SeqIO
from tqdm.contrib.concurrent import process_map

names = set()
with open('tmp/refseq_protein.txt') as f:
    for line in f:
        ls = line.rstrip().split('\t')
        name = ls[2].split(' >')[0].split(' ',1)[1].rsplit(' [')[0].split('MULTISPECIES: ')[-1]
        if 'hypothetical' not in name and ', partial' not in name:
            names.add(name)

In [3]:
rep = set()
ref = set()
rev = defaultdict(set)
tset = set()
with open('sarg_ref.fa') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        name = record.description.split(' >')[0].split(' ',1)[1].rsplit(' [')[0].split('MULTISPECIES: ')[-1]
        rev[name].add('@'.join(record.id.split('|')[1:3]))
        rep.add(record.seq)
        ref.add(record.id.split('|')[-1])
        tset.add('@'.join(record.id.split('|')[1:3]))

rev = {x: ' | '.join(sorted(y)) for x, y in rev.items()}

In [4]:
def reader(file):
    df = pd.read_table(file, header=None, low_memory=False, names= ['accession','name','evidence','symbol','identifier']).fillna('NA')
    df = df[df.name.isin(names)]
    df = df[~df.accession.isin(ref)]
    df = df[~df.accession.isin(df[df.evidence.str.contains('NA|Domain')].accession)]
    df = df[~df.accession.isin(df[(df.evidence.str.contains('TIGR') | (df.identifier.str.contains('PF'))) & (df.symbol == 'NA')].accession)]
    return df

evidence = pd.concat(process_map(reader, glob.glob('tmp/refseq/*.tsv'), bar_format = '{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}', max_workers=48, chunksize=1, leave=False))

  0%|          | 0/781

In [5]:
## bugfix
## https://www.ncbi.nlm.nih.gov/genome/annotation_prok/evidence/NBR012193/
evidence.loc[evidence.evidence == 'NBR012193', 'name'] = 'quaternary ammonium compound efflux SMR transporter EmrC'

In [6]:
evidence['count'] = evidence.groupby(['name', 'evidence']).transform('size')
rmk = evidence[['name', 'evidence', 'symbol', 'identifier', 'count']].drop_duplicates()
rmk['mapped'] = rmk.name.map(rev).fillna('')
rmk['unique'] = (rmk.mapped.str.count('\\|') == 0) & (rmk.mapped != '')
rmk['valid'] = rmk.apply(lambda x: re.sub('^bla|\\(|\\)', '', x.mapped.split('@')[-1].lower()) in re.sub('\\(|\\)', '', x['name'].lower()) if len(x.mapped.split('@')[-1]) > 4 else x.mapped.lower().split('@')[-1] in x['name'].lower(), axis=1)
rmk.loc[rmk.name.str.contains('/'), 'valid'] = False

In [7]:
with open('misc/whitelist.json') as f:
    wl = json.load(f)

rmkk = rmk[~((rmk.unique) & (rmk.valid))].copy()
rmkk['reviewed'] = rmkk.name.apply(lambda x: wl.get(x, {}).get('reviewed', ''))
rmkk = rmkk.sort_values(['name', 'evidence', 'symbol'])

## easy cases
bla = rmkk[(rmkk.symbol.str.contains('^bla', regex=True)) & (rmkk.symbol.str.replace('^bla', '', regex=True) == rmkk.name.str.split(' |-').str.get(0)) & (~rmkk.symbol.str.contains('CMA|CSA|CFE|LAT|LCR|NPS', regex=True))].copy()
bla['gene'] = 'beta-lactam@' + bla.symbol
mcr = rmk[rmk.name.str.contains('^MCR', regex=True)].copy()
mcr['gene'] = 'colistin@' + mcr.name.str.split(' |-related').str.get(0).str.lower()

rmkk = rmkk[~(rmkk.name.isin(bla.name) | (rmkk.name.isin(mcr.name)))]
wl = defaultdict(dict)
for _, i in rmkk.iterrows():
    wl[i['name']]['reviewed'] = i.reviewed
    wl[i['name']]['gene'] = i.mapped
    if 'remark' not in wl[i['name']]:
        wl[i['name']]['remark'] = []
    wl[i['name']]['remark'].append(' | '.join([i.evidence, i.symbol, i.identifier, str(i['count'])]))

wl = {x:y for x,y in wl.items() if x in set(rmkk.name)}
with open('misc/whitelist.json', 'w') as f:
    json.dump(wl, f, sort_keys=True, indent=4)

rev.update(rmkk.set_index('name').reviewed.to_dict())
rev.update(bla.set_index('name').gene.to_dict())
rev.update(mcr.set_index('name').gene.to_dict())

In [8]:
for i,j in rmkk[(rmkk.reviewed == '') | (rmkk.reviewed.str.contains(' '))].groupby(['name', 'mapped', 'reviewed']):
    print(f'{i[0]}')
    print(f'gene: {i[1]}')
    print(f'reviewed: {i[2]}')

    for _, k in j.iterrows():
        print(k.evidence, k.symbol, k.identifier, k['count'])
    print('\n')

print(f'reviewed: {rmkk[rmkk.reviewed != ''].name.nunique()}, unreviewed: {rmkk[rmkk.reviewed == ''].name.nunique()}')
assert sum(rmkk.reviewed == '') == 0, 'Make sure all uncertain cases are reviewed in <misc/whitelist.json>.'

reviewed: 335, unreviewed: 0


In [9]:
def parser(file):
    records = []
    with gzip.open(file, 'rt') as f:
        for record in SeqIO.parse(f, 'fasta'):
            if record.seq not in rep:
                name = record.description.split(' >')[0].split(' ',1)[1].rsplit(' [')[0].split('MULTISPECIES: ')[-1]
                gene = rev.get(name)
                if gene is not None and gene != '' and gene != '*':
                    if record.id in aset:
                        record.id = 'SARG|' + gene.replace('@', '|') + '|' + record.id
                        record.description = record.description.split(' ', 1)[-1].split(' >')[0]
                        records.append(record)
    return records

aset = set(evidence.accession)
r = process_map(parser, glob.glob('tmp/refseq/*.faa.gz'), bar_format = '{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt}', max_workers=48, chunksize=1, leave=False)

  0%|          | 0/781

In [10]:
records = []
ids = set()
sset = set()
for i in r:
    records.extend(i)
    ids.update(set(record.id.split('|')[-1] for record in i))
    sset.update(set('@'.join(record.id.split('|')[1:3]) for record in i))

with open('tmp/sarg_tmp.fa', 'w') as output_handle:
    SeqIO.write(records, output_handle, 'fasta')

evidence['type'] = evidence.name.map(rev).str.split('@').str.get(0).fillna('')
evidence['subtype'] = evidence.name.map(rev).str.split('@').str.get(1).fillna('')
evidence[evidence.accession.isin(ids)].groupby(['type', 'subtype', 'name', 'evidence', 'symbol', 'identifier'], as_index=False).size().rename({'size': 'count'}, axis=1).to_csv('misc/evidence.tsv', sep='\t', index=False)

In [11]:
sset - tset

{"aminoglycoside@aph(3'')*",
 'aminoglycoside@rmt*',
 'beta-lactam@blaACA',
 'beta-lactam@mec*',
 'macrolide-lincosamide-streptogramin@erm*',
 'macrolide-lincosamide-streptogramin@lsa*',
 'macrolide-lincosamide-streptogramin@mph*',
 'macrolide-lincosamide-streptogramin@msr*',
 'macrolide-lincosamide-streptogramin@sal*',
 'macrolide-lincosamide-streptogramin@vat*',
 'macrolide-lincosamide-streptogramin@vga*',
 'nitroimidazole@nim*',
 'rifamycin@rph*',
 'tetracycline@tet*',
 'trimethoprim@dfr*'}